<a href="https://colab.research.google.com/github/hthomas229/PurpleCrown/blob/main/watchdog_yt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Watchdog 🐶

Python's watchdog is a library that monitors a file system for events like creating, deleting, or modifying files and directories. It's used to trigger actions in response to these changes.

Key Concepts
Observers: These are the main objects that run in the background. An observer is responsible for watching one or more directories for changes.

Handlers: These objects define the actions to take when an event occurs. You create a custom handler class with methods like on_created(), on_deleted(), on_modified(), and on_moved() to specify what your program should do.

Events: These are the file system changes that the observer detects. The watchdog library detects four primary event types:

FileCreatedEvent: A new file is added.

FileDeletedEvent: An existing file is removed.

FileModifiedEvent: An existing file's content is changed.

FileMovedEvent: A file or directory is moved or renamed.

Recursive Monitoring: Watchdog can monitor a directory and all of its subdirectories, which is useful for tasks that involve a large project folder.

Install watchdog

In [ ]:
!pip install watchdog

In [ ]:
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time
import os
import pandas as pd

##Create Handler and Observer

In [ ]:
#define folder to watch

WATCH_FOLDER = "watchdog_folder"


#define Event Handler

class FileHandler(FileSystemEventHandler):
  def on_created(self,event):
    if not event.is_directory:
      time.sleep(1)  #Give the file some time to write
      file_name = os.path.basename(event.src_path) # Extract just the filename
      print(f"🔍 Detected file: {file_name}")



if __name__ == "__main__":
    os.makedirs(WATCH_FOLDER, exist_ok=True)


    observer = Observer()
    observer.schedule(FileHandler(), WATCH_FOLDER, recursive=False)
    observer.start()
    print(f"👁️Watching folder: {WATCH_FOLDER}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
        print("🛑 Watchdog stopped!")
    observer.join()

👁️Watching folder: watchdog_folder
🔍 Detected file: wilco.csv
🛑 Watchdog stopped!


##Add Some processing functionality


##Have the folder take in an Excel spreadsehhet and convert all column names to lowercase, create a processed folder and move it there.


In [ ]:
import os
import time
import shutil
import pandas as pd



def process_file(file_path):
    """
    Simulates processing a file, converts Excel headers to lowercase, and moves the file.
    """
    try:
        print(f"⚙️ Processing file: {os.path.basename(file_path)}...")

        # Check if the file is an Excel file before processing headers
        if file_path.endswith(('.xlsx', '.xls')):
            # Read the Excel file into a pandas DataFrame
            df = pd.read_excel(file_path)

            # Convert all column names (headers) to lowercase
            df.columns = df.columns.str.lower()

            # Save the modified DataFrame, overwriting the original file
            df.to_excel(file_path, index=False)
            print("✅ Excel headers converted to lowercase.")
        else:
            print("ℹ️ File is not an Excel file; skipping header conversion.")

        # ... (rest of the code to move the file)
        os.makedirs(PROCESSED_FOLDER, exist_ok=True)
        destination_path = os.path.join(PROCESSED_FOLDER, os.path.basename(file_path))
        shutil.move(file_path, destination_path)

        print(f"✅ Finished processing and moved to: {destination_path}")
        return destination_path

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred during processing: {e}")
        return None

#Modiy our script to accept process_file()

In [ ]:
#define folder to watch

WATCH_FOLDER = "watchdog_folder"
PROCESSED_FOLDER = "processed"  # add folder for processed output


#define Event Handler

class FileHandler(FileSystemEventHandler):
  def on_created(self,event):
    if not event.is_directory:
      time.sleep(1)  #Give the file some time to write
      file_name = os.path.basename(event.src_path) # Extract just the filename
      print(f"🔍 Detected file: {file_name}")
      output_path = process_file(event.src_path)



if __name__ == "__main__":
    os.makedirs(WATCH_FOLDER, exist_ok=True)


    observer = Observer()
    observer.schedule(FileHandler(), WATCH_FOLDER, recursive=False)
    observer.start()
    print(f"👁️Watching folder: {WATCH_FOLDER}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
        print("🛑 Watchdog stopped!")
    observer.join()

👁️Watching folder: watchdog_folder
🔍 Detected file: watch-test.xlsx
⚙️ Processing file: watch-test.xlsx...
✅ Excel headers converted to lowercase.
✅ Finished processing and moved to: processed/watch-test.xlsx
🛑 Watchdog stopped!
